# **Multilingual Sports Commentary**

This notebook demonstrates how to build an end-to-end pipeline for generating, translating, and vocalizing sports commentary using Sarvam AI's suite of models.

### **Use Case**
Automating regional language broadcasts for global sports events. 
1. **Generate:** Use **Sarvam-M** to create high-energy commentary.
2. **Translate:** Use **Mayura:v1** to convert text into multiple Indian languages.
3. **Vocalize:** Use **Bulbul:v3** to generate broadcast-quality audio.

### **1. Prerequisites / Installation**

In [ ]:
# Pinning version for reproducibility as per PR review feedback
!pip install -q sarvamai==0.1.24

### **2. Import Libraries**

In [ ]:
import os
import base64
from sarvamai import SarvamAI
from IPython.display import Audio, display

### **3. API Key Setup**

To use this notebook, obtain an API key from the [Sarvam AI Dashboard](https://dashboard.sarvam.ai).

In [ ]:
SARVAM_API_KEY = os.environ.get("SARVAM_API_KEY", "")
if not SARVAM_API_KEY or SARVAM_API_KEY == "YOUR_SARVAM_API_KEY":
    raise RuntimeError(
        "SARVAM_API_KEY is not set. Add it to your .env file or set the environment variable."
    )

### **4. Client Initialization**

In [ ]:
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

### **5. Parameter Reference**

| Parameter | Type | Required | Options/Range | Description |
| :--- | :--- | :--- | :--- | :--- |
| `input` | string | Yes | Max 1000 chars | The text to be processed. |
| `source_language_code` | string | Yes | en-IN, hi-IN, etc. | BCP-47 language code. |
| `target_language_code` | string | Yes | te-IN, hi-IN, etc. | The output regional language. |
| `speaker` | string | Yes | shubh, priya, etc. | The voice personality for TTS. |
| `model` | string | Yes | sarvam-m, bulbul:v3 | The specific model to invoke. |

### **6. The AI-Powered Sports Pipeline**

#### **Step 6.1: Commentary Generation**

In [ ]:
system_prompt = "You are an enthusiastic football commentator. Generate a short, 1-sentence, high-energy commentary for a spectacular goal scored by Arsenal."

try:
    chat_response = client.chat.completions(
        model="sarvam-m",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "The score is 0-0 in the 90th minute, and Martin Ødegaard just hit a screamer from outside the box!"}
        ]
    )
    
    if not chat_response or not chat_response.choices:
        raise ValueError("Model returned no choices. Check your API quota or content filters.")
        
    generated_commentary = chat_response.choices[0].message.content
    print(f"Generated English Commentary: {generated_commentary}")

except Exception as e:
    print(f"Error during commentary generation: {e}")
    generated_commentary = "Goal for Arsenal! An incredible strike from the captain!" # Safe fallback

#### **Step 6.2: Multilingual Loop (Translate & Speak)**

In [ ]:
target_languages = ["te-IN", "hi-IN", "ta-IN"]

speaker_map = {
    "te-IN": "priya", 
    "hi-IN": "shubh",
    "ta-IN": "kavya"
}

for lang in target_languages:
    try:
        # 1. Translate
        trans_res = client.text.translate(
            input=generated_commentary,
            source_language_code="en-IN",
            target_language_code=lang,
            model="mayura:v1"
        )
        translated_text = trans_res.translated_text
        
        # 2. Text-to-Speech
        tts_res = client.text_to_speech.convert(
            input=translated_text,
            target_language_code=lang,
            model="bulbul:v3",
            speaker=speaker_map.get(lang, "shubh"),
            speech_sample_rate=24000
        )
        
        if not tts_res.audios:
            print(f"No audio returned for {lang}, skipping.")
            continue
            
        # 3. Handle Audio Data
        audio_bytes = base64.b64decode(tts_res.audios[0])
        filename = f"commentary_{lang}.wav"
        with open(filename, "wb") as f:
            f.write(audio_bytes)
            
        print(f"\n--- {lang} Broadcast ---")
        print(f"Text: {translated_text}")
        display(Audio(filename=filename))
        
    except Exception as e:
        print(f"Error processing {lang}: {e}")

### **7. Error Handling**

| Error Code | HTTP Status | Cause | Solution |
| :--- | :--- | :--- | :--- |
| `invalid_api_key_error` | 403 | Invalid API key | Verify your key in the Sarvam Dashboard. |
| `insufficient_quota_error` | 429 | Quota exceeded | Check your usage limits. |
| `internal_server_error` | 500 | Server-side issue | Wait and retry your request. |

### **8. Conclusion & Resources**

This recipe demonstrates how to chain multiple AI capabilities into a single, cohesive media-tech application. 

* [Sarvam AI Docs](https://docs.sarvam.ai)
* [Indic Language Support List](https://docs.sarvam.ai/language-support)

**Keep Building! 🚀**